In [ ]:
seq_0 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_0.csv')
seq_1 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_1.csv')
seq_2 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_2.csv')
seq_3 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_3.csv')
seq_4 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_4.csv')
seq_5 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_5.csv')
seq_6 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_6.csv')
seq_7 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_7.csv')
seq_8 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_8.csv')
seq_9 = pd.read_csv('/media/olafsson/Jupyter1/sequencing/sequencing_working_split_9.csv')
seq_all = pd.concat([seq_0, seq_1, seq_2, seq_3, seq_4, seq_5, seq_6, seq_7, seq_8, seq_9], axis=0)

In [ ]:
grna_df = pd.read_csv('/media/olafsson/Jupyter1/sequencing/orig/grna_sublibrary.csv')
all_df, high_df, low_df = process_barecodes(seq_all)

In [ ]:
all_df = find_grna(df=all_df, grna_df=grna_df)
high_df = find_grna(df=high_df, grna_df=grna_df)
low_df = find_grna(df=low_df, grna_df=grna_df)
all_df = translate_barecodes(all_df, grna_df)
high_df = translate_barecodes(high_df, grna_df)
low_df = translate_barecodes(low_df, grna_df)

In [ ]:
temp = all_df[all_df['count']>=100]
print(len(all_df), len(temp))
temp = temp[temp['grna']!='error']
unique_palte = temp['plate'].value_counts()
unique_row = temp['row'].value_counts()
unique_col = temp['col'].value_counts()
print(unique_palte)
print(unique_row)
print(unique_col)
temp
# aa, tt, cc, gg, at, ta, cg, gc

In [ ]:
temp.to_csv('/mnt/data/CellVoyager/20x/tsg101/crispr_screen/all/measurements/sequencing.csv', index=True)

In [ ]:
temp = high_df[high_df['count']>=100]
unique_palte = temp['plate'].value_counts()
unique_row = temp['row'].value_counts()
unique_col = temp['col'].value_counts()
print(unique_palte)
print(unique_row)
print(unique_col)
temp
# aa, tt, cc, gg, at, ta, cg, gc

In [ ]:
test_error(all_df,min_=25,max_=100)
test_error(low_df,min_=25,max_=100)
test_error(high_df,min_=25,max_=100)

In [ ]:
new_reads_loc = '/media/olafsson/Jupyter1/sequencing/reads_new.csv'
all_df.to_csv(new_reads_loc, index=False, header=True)

In [ ]:
min_count = 25
feature_data = pd.read_csv('/media/olafsson/Data2/methods_paper/data/cell_features.csv')
resnet_loc = '/media/olafsson/Data2/methods_paper/torch_models/new_train/aug/erm/resnet50/139_model98.0230508_.pth_result.csv'
merged_df, well = preprocess_image_data(df=feature_data, resnet_loc=resnet_loc, min_count=min_count, metric='mean', plot=True, score='pc_score')

In [ ]:
df = all_df.copy()
print('1',len(df))
#df['plate'] = df['plate'].replace({'p8': 'p4', 'p4': 'p8'})
#df['prc'] = df['plate']+'_'+df['row']+'_'+df['col']

gene_column = 'grna'
df = df[df['col'] != 'c1']
df = df[df['col'] != 'c2']
df = df[df['col'] != 'c3']
df = df[df['col'] != 'c15']
df = df[df['col'] != 'c12']

print('2',len(df))

df = df[~df.apply(lambda row: row.astype(str).str.contains('error').any(), axis=1)]
print('3',len(df))
plates = ['p1','p2','p3','p4']
df = df[df['plate'].isin(plates)]
print('4',len(df))
multi_target_grnas = pd.read_csv('/media/olafsson/Jupyter1/sequencing/orig/gRNAs_multitargets_Sublibrary1.csv')
mtg = multi_target_grnas['gene'].tolist()
df = df[~df['gene'].isin(mtg)]
df = df[df['count']>=25]

g14 = df[df['gene'].str.contains('239740')]
r18 = df[df['gene'].str.contains('205250')]

#gra14_wells = g14['prc'].unique().tolist()
#df = df[df['prc'].isin(gra14_wells)]
print('5',len(df))

print('screen coverage', len(df), len(df)/(4*384))

independent_variables = generate_fraction_map(df=df, gene_column=gene_column,  min_=0.05, metric='fraction', plot=True)

iv_col_loc = '/media/olafsson/Data2/methods_paper/data/iv_col.csv'
all_df.to_csv(iv_col_loc)

iv_loc = '/media/olafsson/Data2/methods_paper/data/iv.csv'
independent_variables.to_csv(iv_loc)

iv_loc = '/media/olafsson/Data2/methods_paper/data/iv.csv'
dv_loc = '/media/olafsson/Data2/methods_paper/data/dv.csv'
df, res = mixed_model(iv_loc,dv_loc)
print(res)
hits = df[df['P>|z|']<=0.05]
low_hits = hits[hits['Coef.']<=-0.25]
high_hits = hits[hits['Coef.']>=0.25]
high_hits

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5))
for i,val in enumerate(low_ls):
    temp = merged_df[merged_df['prc']==val]
    sns.histplot(data=temp, x='recruitment', kde=False, stat='density', binwidth=0.1,element="step", ax=ax[0], color='teal', log_scale=False)
    sns.histplot(data=temp, x='pc_score', kde=False, stat='density', binwidth=0.1,element="step", ax=ax[1], color='teal', log_scale=False)
    #sns.kdeplot(data=temp, x='recruitment', ax=ax[0], color='teal')
    #sns.kdeplot(data=temp, x='pc_score', ax=ax[1], color='teal')
    #ax.set_xscale('log')
    ax[0].set_title('Low resnet score')
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].set_xlim([0, 4])
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].set_xlim([0, 1])

In [ ]:
#define function to calculate Gini coefficient
def gini(x):
    #0 represents perfect income equality (everyone has the same income)
    #1 represents perfect income inequality (one individual has all the income)
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

df = all_df.copy()
df = df[~df.apply(lambda row: row.astype(str).str.contains('error').any(), axis=1)]
df = df[df['count']>=25]
prc_ls = []
gini_ls = []
for i,val in enumerate(df['prc'].unique().tolist()):
    temp = df[df['prc']==val]
    x=temp['count'].to_numpy()
    gini_val = gini(x)
    prc_ls.append(val)
    gini_ls.append(gini_val)
gini_df = pd.DataFrame()
gini_df['prc'] = prc_ls
gini_df['gini'] = gini_ls
gini_df = gini_df[gini_df['gini']>0]
balanced_wells = gini_df['prc'].unique().tolist()
gini_df = gini_df.sort_values(by = ['gini'], ascending = [True], na_position = 'first')
fig, ax = plt.subplots(1,2,figsize=(10,5))
sns.histplot(data=gini_df, x='gini', kde=False, stat='count', binwidth=0.01,element="step", ax=ax[0], color='teal', log_scale=False)

In [ ]:
df = all_df.copy()

print('1',len(df))
#df['plate'] = df['plate'].replace({'p1': 'p4', 'p4': 'p1'})
#df['prc'] = df['plate']+'_'+df['row']+'_'+df['col']

gene_column = 'gene'
df = df[df['col'] != 'c1']
df = df[df['col'] != 'c2']
df = df[df['col'] != 'c3']
df = df[df['col'] != 'c15']
df = df[df['col'] != 'c12']

print('2',len(df))

df = df[~df.apply(lambda row: row.astype(str).str.contains('error').any(), axis=1)]
print('3',len(df))
plates = ['p1','p2','p3','p4']
df = df[df['plate'].isin(plates)]
df = df[df['prc'].isin(balanced_wells)]

multi_target_grnas = pd.read_csv('/media/olafsson/Jupyter1/sequencing/orig/gRNAs_multitargets_Sublibrary1.csv')
mtg = multi_target_grnas['gene'].tolist()
df = df[~df['gene'].isin(mtg)]

gra14_wells = g14['prc'].unique().tolist()
df = df[df['prc'].isin(gra14_wells)]

print('4',len(df))
g14 = df[df['gene'].str.contains('239740')]
r18 = df[df['gene'].str.contains('205250')]

print('5',len(df))

print('screen coverage', len(df), len(df)/(4*384))
df = df[df['count']>=50]
independent_variables = generate_fraction_map(df=df, gene_column=gene_column,  min_=0.05, metric='fraction', plot=True)

iv_col_loc = '/media/olafsson/Data2/methods_paper/data/iv_col.csv'
all_df.to_csv(iv_col_loc)

iv_loc = '/media/olafsson/Data2/methods_paper/data/iv.csv'
independent_variables.to_csv(iv_loc)

iv_loc = '/media/olafsson/Data2/methods_paper/data/iv.csv'
dv_loc = '/media/olafsson/Data2/methods_paper/data/dv.csv'

df, res = reg_model(iv_loc, dv_loc)

hits = df[df['P>|t|']<=0.05]
low_hits = hits[hits['coef']<=-0.25]
high_hits = hits[hits['coef']>=0.25]
high_hits